#### Prac 2: Implement Decision Tree Algorithm with Post Pruning

In [1]:
import pandas as pd
import pprint
import numpy as np

In [2]:
df = pd.read_csv("restaurant.csv")
df

,Alt,Bar,Fri,Hun,Pat,Price,Rain,Res,Type,Est,Wait
0,Yes,No,No,Yes,Some,3,No,Yes,French,0-10,Yes
1,Yes,No,No,Yes,Full,1,No,No,Thai,30-60,No
2,No,Yes,No,No,Some,1,No,No,Burger,0-10,Yes
3,Yes,No,Yes,Yes,Full,1,No,No,Thai,Oct-30,Yes
4,Yes,No,Yes,No,Full,3,No,Yes,French,>60,No
5,No,Yes,No,Yes,Some,2,Yes,Yes,Italian,0-10,Yes
6,No,Yes,No,No,None,1,Yes,No,Burger,0-10,No
7,No,No,No,Yes,Some,2,Yes,Yes,Thai,0-10,Yes
8,No,Yes,Yes,No,Full,1,Yes,No,Burger,>60,No
9,Yes,Yes,Yes,Yes,Full,3,No,Yes,Italian,Oct-30,No


In [3]:
def calculate_entropy(df_label):
    classes,class_counts = np.unique(df_label,return_counts = True)
    entropy_value = np.sum([(-class_counts[i]/np.sum(class_counts))*np.log2(class_counts[i]/np.sum(class_counts)) 
                        for i in range(len(classes))])
    return entropy_value

In [4]:
def calculate_information_gain(dataset,feature,label): 
    # Calculate the dataset entropy
    dataset_entropy = calculate_entropy(dataset[label])   
    values,feat_counts= np.unique(dataset[feature],return_counts=True)
    weighted_feature_entropy = np.sum([(feat_counts[i]/np.sum(feat_counts))*calculate_entropy(dataset.where(dataset[feature]==values[i]).dropna()[label]) for i in range(len(values))])    
    feature_info_gain = dataset_entropy - weighted_feature_entropy
    return feature_info_gain

In [5]:
def create_decision_tree(dataset,df,features,label,parent):
    datum=np.unique(df[label],return_counts=True)
    unique_data=np.unique(dataset[label])
    
    if len(unique_data)<=1:
        return unique_data[0]
    elif len(dataset)==0:
        return unique_data[np.argmax(datum[1])]
    elif len(features)==0:
        return parent
    else:
        parent=unique_data[np.argmax(datum[1])]
        item_values=[calculate_information_gain(dataset,feature,label)for feature in features]
        optimum_feature_index = np.argmax(item_values)
        optimum_feature = features[optimum_feature_index]
        decision_tree  = {optimum_feature:{}}
        features=[i for i in features if i != optimum_feature]
        for value in np.unique(dataset[optimum_feature]):
            min_data=dataset.where(dataset[optimum_feature]== value).dropna()
            min_tree=create_decision_tree(min_data,df,features,label,parent)
            decision_tree[optimum_feature][value]=min_tree
        return(decision_tree)

In [6]:
features = df.columns[:-1]
label = 'Wait'
parent=None
features

Index(['Alt', 'Bar', 'Fri', 'Hun', 'Pat', 'Price', 'Rain', 'Res', 'Type',
       'Est'],
      dtype='object')

In [7]:
decision_tree = create_decision_tree(df,df,features,label,parent)

In [8]:
pprint.pprint(decision_tree)

{'Pat': {'Full': {'Hun': {'No': 'No',
                          'Yes': {'Type': {'Burger': 'Yes',
                                           'Italian': 'No',
                                           'Thai': {'Fri': {'No': 'No',
                                                            'Yes': 'Yes'}}}}}},
         'None': 'No',
         'Some': 'Yes'}}
